## Export an Annotation Data and split status.

In [87]:
import json
from PIL import Image
from matplotlib import pyplot as plt
import pandas as pd
import sys
import ast
import shutil

### 1. Fetch DataSet
---

In [8]:
# テキストファイルのパスを指定します
file_path_infos = [{"file_path":"dataset/findit2/train.txt", "data_name": "train"},
                  {"file_path":"dataset/findit2/test.txt", "data_name": "test"},
                  {"file_path":"dataset/findit2/val.txt", "data_name": "val"}
                 ]

# タブ区切りの場合
datas = []
for file_path_info in file_path_infos:
    datas.append(pd.read_csv(file_path_info["file_path"], delimiter=','))

In [35]:
datasets = {}
for df, info in zip(datas, file_path_infos):
    df_having_annotations = df[df["forgery annotations"]!="0"]
    df_not_having_annotations = df[df["forgery annotations"]=="0"]
    
    datasets[info["data_name"]] = {
        "hv_annotations": df_having_annotations,
        "not_hv_annotations": df_not_having_annotations
    }
    
    """
    show data records:
    """
    print("■" + info["data_name"])
    print("Having annotations count : " + str(len(df_having_annotations)))
    print("Not Having annotations count : " + str(len(df_not_having_annotations)))

■train
Having annotations count : 94
Not Having annotations count : 483
■test
Having annotations count : 35
Not Having annotations count : 183
■val
Having annotations count : 34
Not Having annotations count : 159


#### Show DataFrame Sample

In [28]:
datasets["train"]["hv_annotations"]

,image,digital annotation,handwritten annotation,forged,forgery annotations
0,X00016469622.png,1,1,1,"{'filename': 'X00016469622.png', 'size': 23072..."
6,X51005230617.png,0,1,1,"{'filename': 'X51005230617.png', 'size': 79672..."
25,X51005361906.png,0,1,1,"{'filename': 'X51005361906.png', 'size': 38813..."
30,X51005361946.png,0,1,1,"{'filename': 'X51005361946.png', 'size': 77602..."
32,X51005365179.png,1,0,1,"{'filename': 'X51005365179.png', 'size': 40184..."
...,...,...,...,...,...
557,X51008142031.png,0,0,1,"{'filename': 'X51008142031.png', 'size': 64162..."
560,X51008142063.png,0,0,1,"{'filename': 'X51008142063.png', 'size': 69819..."
569,X51008164998.png,0,0,1,"{'filename': 'X51008164998.png', 'size': 13113..."
570,X51009447842.png,0,0,1,"{'filename': 'X51009447842.png', 'size': 67002..."


In [93]:
datasets["train"]["not_hv_annotations"]

,image,digital annotation,handwritten annotation,forged,forgery annotations
1,X00016469623.png,1,1,0,0
2,X00016469670.png,1,1,0,0
3,X00016469671.png,1,1,0,0
4,X00016469672.png,1,1,0,0
5,X51005200938.png,0,1,0,0
...,...,...,...,...,...
571,X51009453729.png,1,1,0,0
572,X51009453801.png,1,0,0,0
573,X51009453804.png,1,1,0,0
574,X51009568881.png,0,0,0,0


In [80]:
datasets["train"]["hv_annotations"].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94 entries, 0 to 575
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   image                   94 non-null     object
 1   digital annotation      94 non-null     int64 
 2   handwritten annotation  94 non-null     int64 
 3   forged                  94 non-null     int64 
 4   forgery annotations     94 non-null     object
dtypes: int64(3), object(2)
memory usage: 6.5+ KB


### 2. Move Dataset
---

In [88]:
def move_file(source_path, destination_path):
    try:
        shutil.move(source_path, destination_path)
        print("ファイルが移動されました。")
    except FileNotFoundError:
        print("移動元のファイルが見つかりません。")
    except PermissionError:
        print("移動元または移動先のファイルにアクセス権がありません。")
    except Exception as e:
        print(f"エラーが発生しました: {e}")

In [98]:
dataset_dir = "dataset/findit2/test/"
list_filename = list(datasets["test"]["hv_annotations"]["image"])
list_is_forged = list(datasets["test"]["hv_annotations"]["forged"])

for filename, is_forged in zip(list_filename, list_is_forged):
    source_path = dataset_dir + filename
    if is_forged == 1:
        destination_path = dataset_dir + "/forged/" + filename
    else:
        destination_path = dataset_dir + "/not_forged/" + filename
    print(source_path)
    move_file(source_path, destination_path)

dataset/findit2/test/X51005230616.png
ファイルが移動されました。
dataset/findit2/test/X51005433556.png
ファイルが移動されました。
dataset/findit2/test/X51005442361.png
ファイルが移動されました。
dataset/findit2/test/X51005447851.png
ファイルが移動されました。
dataset/findit2/test/X51005447853.png
ファイルが移動されました。
dataset/findit2/test/X51005568866.png
ファイルが移動されました。
dataset/findit2/test/X51005605295.png
ファイルが移動されました。
dataset/findit2/test/X51005676541.png
ファイルが移動されました。
dataset/findit2/test/X51005676546.png
ファイルが移動されました。
dataset/findit2/test/X51005705729.png
ファイルが移動されました。
dataset/findit2/test/X51005711402.png
ファイルが移動されました。
dataset/findit2/test/X51005757243.png
ファイルが移動されました。
dataset/findit2/test/X51005757349.png
ファイルが移動されました。
dataset/findit2/test/X51006008206.png
ファイルが移動されました。
dataset/findit2/test/X51006329183.png
ファイルが移動されました。
dataset/findit2/test/X51006388044.png
ファイルが移動されました。
dataset/findit2/test/X51006401977.png
ファイルが移動されました。
dataset/findit2/test/X51006414715.png
ファイルが移動されました。
dataset/findit2/test/X51006556603.png
ファイルが移動されました。
dataset/find